## OpenAI Gym Warm-Up

In [1]:
# Import Environment class and Libraries
from frozen_lake import FrozenLakeEnv
import numpy as np
import sys
import matplotlib.pyplot as plt
import time


# Create Environment Object
env = FrozenLakeEnv(map_name ="4x4", is_slippery=False)


# Access the number of states:
nS = env.observation_space
print("State space of the Env: ", nS)

# or you could even use 
nS = env.nS
print("State space of the Env by accessing env.nS: ", nS)


# Action space of the agent:
nA = env.nA
print("Action space of the Env: ", nA)

State space of the Env:  16
State space of the Env by accessing env.nS:  16
Action space of the Env:  4


In [2]:
"""
For policy iteration, you would need to access
State(s), Action(a), Next State(ns), Reward(r), episode ended? (is_done) tuples.

Note that in this environment, the orientation of the agent does not matter.
No matter what direction the agent is facing, if, say a left action is performed, 
the agent moves to the left of the crrent state.
"""

# For actions, this is the corresponding dictionary:
action_names = {0:'L', 1:'D', 2:"R", 3:"U"}

"""
Here, 
'L' means left
'D' means down
'R' means right
'U' means up



You can access these tuples by simply env.P[s][a].
where 's' is state, and 'a' is action. For example, let's say we are at state '4',
and we take an action '1' or "Down". The next state (ns) would be 8, the episode would not have ended (is_done), 
the reward (r) is 0 and the transition probabilty (prob) is 1 because this is a deterministic setting.
"""

prob, ns, r, is_done = env.P[4][1][0]


print("Transition Probabilty: ", prob)
print("Next State: ", ns)
print("Reward: ", r)
print("Episode ended? : ", is_done)
# Note that we need to add a [0] after env.P[s][a] because it returns a list containing the tuple

Transition Probabilty:  1.0
Next State:  8
Reward:  0.0
Episode ended? :  False


## Policy Iteration 

- Follow the pseudo-code given in the handout for this section

In [3]:
def print_policy(policy, action_names, states):
    """Print the policy in human-readable format.
    If you've implemented this correctly, the output (for 4x4 map) should be:
    
    D R D L 
    D L D L 
    R D D L 
    L R R L 
    
    Parameters
    ----------
    policy: np.ndarray
        Array of state to action number mappings
    action_names: dict
        Mapping of action numbers to characters representing the action.
    num_states: int
        Number of states in the FrozenLakeEnvironment (16 or 64 for 4x4 or 8x8 maps respectively)      
    """
    
    # WRITE YOUR CODE HERE:
    side = int(np.sqrt(states))
    arr = np.chararray((side, side), unicode = 'True')
    for i in range(side):
        test = np.chararray(side)
        for j in range(side):
            test[j] = action_names[policy[i*side+j]]
        arr[i,:] = test
    print(arr)
        

In [4]:
def evaluate_policy_sync(env, gamma, policy, value_func, max_iterations=int(1e3), tol=1e-3):
    """Performs policy evaluation.
    
    Evaluates the value of a given policy.

    Parameters
    ----------
    env: Frozen Lake Environment
      The environment to compute value iteration for.
    gamma: float
      Discount factor, must be in range [0, 1)
    policy: np.array
      The policy to evaluate. Maps states to actions.
    value_func: np.array
      Array of scalar values for each state
    max_iterations: int
      The maximum number of iterations to run before stopping.
    tol: float
      Determines when value function has converged.

    Returns
    -------
    np.ndarray, int
      The value for the given policy and the number of iterations till
      the value function converged.
    """

    val_iter = 0
    # WRITE YOUR CODE HERE:
    delta = 0
    for i in range(max_iterations):
        for s in range(env.nS):
            a = policy[s]
            _, ns, r, _ = env.P[s][a][0]
            v = r + gamma*(value_func[int(ns)])
            delta = max(delta, np.abs(v - value_func[s]))
            value_func[s] = v
        val_iter += 1
        if delta < tol:
            break
    return value_func, val_iter

In [5]:
def improve_policy(env, gamma, value_func, policy):
    """Performs policy improvement.
    
    Given a policy and value function, improves the policy.

    Parameters
    ----------
    env: Frozen Lake Environment
      The environment to compute value iteration for.
    gamma: float
      Discount factor, must be in range [0, 1)
    value_func: np.ndarray
      Value function for the given policy.
    policy: dict or np.array
      The policy to improve. Maps states to actions.

    Returns
    -------
    bool, np.ndarray
      Returns the new imporved policy.
    """
    
    # WRITE YOUR CODE HERE:
    new_policy = np.zeros(env.nS)
    for s in range(env.nS):
        a_val = np.zeros(env.nA)
        for a in range(env.nA):
            _, ns, r, _ = env.P[s][a][0]
            a_val[a] = r + gamma*(value_func[ns])
        new_policy[s] = np.argmax(a_val)
    return new_policy

In [6]:
def policy_iteration_sync(env, gamma, max_iterations=int(1e3), tol=1e-3):
    """Runs policy iteration.

    See page 85 of the Sutton & Barto Second Edition book.

    You should call the improve_policy() and evaluate_policy_sync() methods to
    implement this method.
    
    If you've implemented this correctly, it should take much less than 1 second.
    
    Parameters
    ----------
    env: Frozen Lake Environment
      The environment to compute value iteration for.
    gamma: float
      Discount factor, must be in range [0, 1)
    max_iterations: int
      The maximum number of iterations to run before stopping.
    tol: float
      Determines when value function has converged.

    Returns
    -------
    (np.ndarray, np.ndarray, int, int)
       Returns optimal policy, value function, number of policy
       improvement iterations, and number of value iterations.
    """
    
    policy = np.random.randint(0, 4, size=env.nS)   #Define random policy
    value_func_init = np.zeros(env.nS)    # Define initial value function
    num_pol_iter = 0
    num_val_iter = 0
    value_func_list = []
    # WRITE YOUR CODE HERE:
    for i in range(max_iterations):
        val_func, val_iter = evaluate_policy_sync(env, gamma, policy, value_func_init)
        num_val_iter += 1
        new_policy = improve_policy(env, gamma, val_func, policy)
        num_pol_iter += 1
        value_func_list.append(val_func)
        if np.array_equal(policy, new_policy):
            break
        policy = new_policy
        value_func_init = val_func
    return policy, value_func_init, num_pol_iter, num_val_iter,value_func_list

In [7]:
def main():
    
    # WRITE YOUR CODE HERE:
    print("RESULTS FOR 4x4 FROZEN LAKE")
    start = time.time()
    env = FrozenLakeEnv(map_name ="4x4", is_slippery=False)
    pol, val_func, pol_it, val_it, val_list = policy_iteration_sync(env, 0.9)
    arr = print_policy(pol, action_names, 16)
    end = time.time()
    print("Time elapsed for 4x4 map: {} seconds".format(np.round(end-start, 4)))
    print("Number of policy improvement steps: {} steps".format(pol_it))
    print("Number of policy evaluation stpes: {} steps".format(val_it))
    
    print("")
    
    print("RESULTS FOR 8x8 FROZEN LAKE")
    start = time.time()
    env = FrozenLakeEnv(map_name ="8x8", is_slippery=False)
    pol, val_func, pol_it, val_it, val_list = policy_iteration_sync(env, 0.9)
    arr = print_policy(pol, action_names, 64)
    end = time.time()
    print("Time elapsed for 4x4 map: {} seconds".format(np.round(end-start, 4)))
    print("Number of policy improvement steps: {} steps".format(pol_it))
    print("Number of policy evaluation stpes: {} steps".format(val_it))
    

    
if __name__ == "__main__":
    main()

RESULTS FOR 4x4 FROZEN LAKE
[['D' 'R' 'D' 'L']
 ['D' 'L' 'D' 'L']
 ['R' 'D' 'D' 'L']
 ['L' 'R' 'R' 'L']]
Time elapsed for 4x4 map: 0.3823 seconds
Number of policy improvement steps: 7 steps
Number of policy evaluation stpes: 7 steps

RESULTS FOR 8x8 FROZEN LAKE
[['D' 'D' 'D' 'D' 'D' 'D' 'D' 'D']
 ['D' 'D' 'D' 'R' 'D' 'D' 'D' 'D']
 ['D' 'D' 'D' 'L' 'D' 'R' 'D' 'D']
 ['R' 'R' 'R' 'R' 'D' 'L' 'D' 'D']
 ['R' 'R' 'U' 'L' 'D' 'D' 'R' 'D']
 ['D' 'L' 'L' 'R' 'R' 'D' 'L' 'D']
 ['D' 'L' 'R' 'U' 'L' 'D' 'L' 'D']
 ['R' 'R' 'U' 'L' 'R' 'R' 'R' 'L']]
Time elapsed for 4x4 map: 3.6084 seconds
Number of policy improvement steps: 14 steps
Number of policy evaluation stpes: 14 steps
